## Applications and limitations of NER models

In this project, we will explore the applications of Entity Named Recognition.

In [ ]:
pip install spacy

Application of Name Entity Recognition

### Use case 1: Search Engine Optimization

In [ ]:
import spacy
from spacy import displacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")
# News
text = """Tesla founded in 2003 by Elon Musk,
         stands as a beacon of innovation in the electric vehicle industry. With Gigafactories
         strategically located in Shanghai, Berlin, and Austin, Tesla continues to redefine mobility.
         Tesla has a market cap of $758.29 Billion."""
# Process the social media text using spaCy
doc = nlp(text)
# Visualize the entities using displaCy (render for static visualization)
options = {"ents": ["PERSON", "ORG", "GPE", "PRODUCT", "MONEY",  "DATE"],
           "colors": {"PERSON": "lightblue", "ORG": "lightgreen", "GPE": "lightcoral",
                      "PRODUCT": "gold", "MONEY": "pink", "DATE": "lightyellow"}}
displacy.render(doc, style="ent", options=options, jupyter=True)

### Use case 2: Social Media Content Curation

In [ ]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")
# Social media content example
social_media_text = """

The new iPhone 15 is finally here! 📱 Can't wait to get my hands on it. #AppleEvent #iPhone15 #tech
Exploring beautiful beaches in Bali 🏖️ #TravelGoals #Bali #Wanderlust
Had a delicious dinner at @LePapillonResto last night. Highly recommend their seafood dishes! #Foodie #RestaurantRecommendation
"""
# Process the social media text using spaCy
doc = nlp(social_media_text)
# Visualize the entities using displaCy (render for static visualization)
displacy.render(doc, style="ent",  jupyter=True)


### Overcoming the limits of Named Entity Recognition

Before Adding a rule to spaCy model

In [ ]:
# Before adding the rule to the model
text = """I love working with Python, it's so versatile.
        Just wrote a script in R as well."""
# Process the unstructured text using spaCy
doc = nlp(text)
# Visualize the entities using displaCy
displacy.render(doc, style="ent", jupyter=True)

Spacy was not able to recognize the programming language. Create a new rule that recognizes "Programming Language" in an unstructured text.

In [ ]:
import spacy
from spacy.tokens import Doc, Span
# Define a custom rule for programming languages
@spacy.Language.component("programming_language_component")
def programming_language_component(doc):
    programming_languages = ["Python", "R", "Java", "C++", "JavaScript", "Ruby"]
    for token in doc:
        if token.text in programming_languages:
            span = Span(doc, token.i, token.i + 1, label="PROG_LANG")
            doc.ents = list(doc.ents) + [span]
    return doc

# Add the custom component to the spaCy pipeline
nlp.add_pipe("programming_language_component", after="ner")

<function __main__.programming_language_component(doc)>

Now, Check if the model Spacy recognize the entities.

In [ ]:
# After adding the rule to the model
text = """I love working with Python, it's so versatile.
        Just wrote a script in R as well."""
# Process the unstructured text using spaCy
doc = nlp(text)
# Visualize the entities using displaCy
displacy.render(doc, style="ent", jupyter=True)

Rules are a powerful way to improve the Spacy model and overcome its limits.

After exploring the applications and limitations of NER models, let's get a metadata from New York Times articles. The goal is to answer the research question:

Geographical Representation in NYT Articles: This research question aims to explore how The New York Times (NYT) covers different regions of the world.




## Get the data from New York Times API

Please obtain your New Times API keys to reproduce this code. I've hidden my API keys for security. https://developer.nytimes.com/docs/archive-product/1/overview

The Archive API returns an array of NYT articles for a given month, going back to 1851. Its response fields are the same as the Article Search API. The Archive API is very useful if you want to build your own database of NYT article metadata.

To answer the first research question, Let's download the metadata of 2023 from January to November.

- The challenge was that when we tried to get all data at the same time, it wouldn't work. After looking up the issue, making multiple requests in quick succession is the reason why we couldn't get the full year data at once.

- We introduce delays of 10s between requests using time.sleep() to avoid overwhelming the server... However it didn't work

- Our last option was to get the data chunk by chunck and combine them all.

- Let's get the data for January to May.

In [ ]:
import requests
import json
import pandas as pd
# Set your API key and the year for which you want to fetch data
api_key = 'KgkYJkjApX9MFWRi3DU0K5doqwXPZLij'
year = 2023
# Initialize an empty DataFrame to store the data
Jan_May = pd.DataFrame()
# Iterate over the months from January to May
for month in range(1, 6):
    api_endpoint = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json"
    request_url = f"{api_endpoint}?api-key={api_key}"
    response = requests.get(request_url)
    data = response.json()
    articles = data.get("response", {}).get("docs", [])
    df = pd.DataFrame(articles)
    Jan_May = pd.concat([Jan_May, df], ignore_index=True)

# Convert 'pub_date' to datetime format
Jan_May['pub_date'] = pd.to_datetime(Jan_May['pub_date'])


Now, let's get the data for June to September.

In [ ]:
# Initialize an empty DataFrame to store the data
June_Sept = pd.DataFrame()
# Iterate over the months from June to September
for month in range(6,10):
    api_endpoint = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json"
    request_url = f"{api_endpoint}?api-key={api_key}"
    response = requests.get(request_url)
    data = response.json()
    articles = data.get("response", {}).get("docs", [])
    df = pd.DataFrame(articles)
    June_Sept = pd.concat([June_Sept, df], ignore_index=True)

# Convert 'pub_date' to datetime format
June_Sept['pub_date'] = pd.to_datetime(June_Sept['pub_date'])


Let's get the data for October to November.

In [ ]:
# Initialize an empty DataFrame to store the data
Oct_Nov = pd.DataFrame()
# Iterate over the months from October to November
for month in range(10,12):
    api_endpoint = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json"
    request_url = f"{api_endpoint}?api-key={api_key}"
    response = requests.get(request_url)
    data = response.json()
    articles = data.get("response", {}).get("docs", [])
    df = pd.DataFrame(articles)
    Oct_Nov = pd.concat([Oct_Nov, df], ignore_index=True)

# Convert 'pub_date' to datetime format
Oct_Nov['pub_date'] = pd.to_datetime(Oct_Nov['pub_date'])


Now, we have the data from January to November. We can concatenate the 3 dataframe into one single dataframe


In [ ]:
# Concatenate The data
full_year_data = pd.concat([Jan_May, June_Sept, Oct_Nov], ignore_index=True)

The column 'pub_date' in the data frame represents the date and time the articles were written. Let's check if the column contains months from January to November.

In [ ]:
full_year_data['pub_date']

0       2023-01-01 00:05:59+00:00
1       2023-01-01 00:15:57+00:00
2       2023-01-01 01:27:54+00:00
3       2023-01-01 04:00:10+00:00
4       2023-01-01 05:06:06+00:00
                   ...           
42513   2023-11-30 16:38:36+00:00
42514   2023-11-30 16:39:28+00:00
42515   2023-11-30 16:43:33+00:00
42516   2023-11-30 16:57:57+00:00
42517   2023-11-30 17:04:11+00:00
Name: pub_date, Length: 42518, dtype: datetime64[ns, UTC]

Display the number of columns and rows in this dataset.

In [ ]:
full_year_data.shape

(42518, 20)

Display the first few rows in the dataset

In [ ]:
full_year_data.head()

,abstract,web_url,snippet,lead_paragraph,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,print_section,print_page,subsection_name
0,Underlying the debate in America over abortion...,https://www.nytimes.com/2022/12/31/us/life-per...,Underlying the debate in America over abortion...,In the months since the Supreme Court overturn...,The New York Times,[],{'main': 'How Do You Think About the Question ...,"[{'name': 'subject', 'value': 'Abortion', 'ran...",2023-01-01 00:05:59+00:00,article,National,U.S.,"{'original': 'By The New York Times', 'person'...",News,nyt://article/db090671-cf4f-58ac-96ca-201b66e5...,83,nyt://article/db090671-cf4f-58ac-96ca-201b66e5...,NaN,NaN,NaN
1,The question is at the heart of the human expe...,https://www.nytimes.com/interactive/2022/12/31...,The question is at the heart of the human expe...,The question is at the heart of the human expe...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'When Does Life Begin?', 'kicker': No...","[{'name': 'subject', 'value': 'Pregnancy and C...",2023-01-01 00:15:57+00:00,multimedia,U.S.,U.S.,{'original': 'By Elizabeth Dias and Bethany Mo...,Interactive Feature,nyt://interactive/6586aa05-82c8-5b1c-981e-e1a1...,0,nyt://interactive/6586aa05-82c8-5b1c-981e-e1a1...,A,1,NaN
2,Texas Christian returned two interceptions for...,https://www.nytimes.com/2022/12/31/sports/ncaa...,Texas Christian returned two interceptions for...,"GLENDALE, Ariz. — Sonny Dykes is not the ideal...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'T.C.U. Upends Michigan in Wild Playo...,"[{'name': 'subject', 'value': 'Football (Colle...",2023-01-01 01:27:54+00:00,article,Sports,Sports,"{'original': 'By Billy Witz', 'person': [{'fir...",News,nyt://article/d25d2952-0afa-59b4-9727-95ada545...,1391,nyt://article/d25d2952-0afa-59b4-9727-95ada545...,A,30,College Football
3,Hours before Russian troops began withdrawing ...,https://www.nytimes.com/2022/12/31/world/europ...,Hours before Russian troops began withdrawing ...,"BUCHA, Ukraine — On one of the last nights of ...",The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'In Bucha, a Final Rampage Served as ...","[{'name': 'glocations', 'value': 'Bucha (Ukrai...",2023-01-01 04:00:10+00:00,article,Foreign,World,{'original': 'By Carlotta Gall and Oleksandr C...,News,nyt://article/20f32fec-caf8-5bde-afb3-b2944903...,1511,nyt://article/20f32fec-caf8-5bde-afb3-b2944903...,A,9,Europe
4,Georgia scored in the final minute of the game...,https://www.nytimes.com/2023/01/01/sports/ncaa...,Georgia scored in the final minute of the game...,ATLANTA — The Georgia Bulldogs appeared very n...,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Georgia Downs Ohio State in Semifina...,"[{'name': 'subject', 'value': 'Football (Colle...",2023-01-01 05:06:06+00:00,article,Sports,Sports,"{'original': 'By Alan Blinder', 'person': [{'f...",News,nyt://article/3ea75071-7052-5fdc-afb1-3ca38638...,947,nyt://article/3ea75071-7052-5fdc-afb1-3ca38638...,D,3,College Football


## Data Transformation

We observe that some columns in the data frame are collections of columns. Let'transform the data for the "keywords" columns form


In [ ]:
# Create a new DataFrame by unnesting the 'keywords' column
keywords_df = pd.json_normalize(full_year_data['keywords'])
# Visualize the first few rows
keywords_df.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,"{'name': 'subject', 'value': 'Abortion', 'rank...","{'name': 'subject', 'value': 'internal-open-ac...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'name': 'subject', 'value': 'Pregnancy and Ch...","{'name': 'subject', 'value': 'Embryos (Human)'...","{'name': 'subject', 'value': 'Biology and Bioc...","{'name': 'subject', 'value': 'Anatomy and Phys...","{'name': 'subject', 'value': 'Religion and Bel...","{'name': 'subject', 'value': 'Abortion', 'rank...","{'name': 'glocations', 'value': 'United States...","{'name': 'subject', 'value': 'Philosophy', 'ra...","{'name': 'subject', 'value': 'internal-sub-onl...",None,...,None,None,None,None,None,None,None,None,None,None
2,"{'name': 'subject', 'value': 'Football (Colleg...","{'name': 'organizations', 'value': 'Big 12 Con...","{'name': 'organizations', 'value': 'Big Ten Co...","{'name': 'organizations', 'value': 'University...","{'name': 'organizations', 'value': 'Texas Chri...","{'name': 'persons', 'value': 'Dykes, Daniel (S...","{'name': 'persons', 'value': 'Duggan, Max (200...","{'name': 'persons', 'value': 'Harbaugh, Jim', ...",None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'name': 'glocations', 'value': 'Bucha (Ukrain...","{'name': 'subject', 'value': 'War Crimes, Geno...","{'name': 'subject', 'value': 'Civilian Casualt...","{'name': 'subject', 'value': 'Russian Invasion...","{'name': 'subject', 'value': 'Defense and Mili...","{'name': 'persons', 'value': 'Putin, Vladimir ...","{'name': 'glocations', 'value': 'Kyiv (Ukraine...","{'name': 'glocations', 'value': 'Russia', 'ran...",None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'name': 'subject', 'value': 'Football (Colleg...","{'name': 'subject', 'value': 'College Football...","{'name': 'subject', 'value': 'Playoff Games', ...","{'name': 'subject', 'value': 'PEACH BOWL (FOOT...","{'name': 'organizations', 'value': 'University...","{'name': 'organizations', 'value': 'Ohio State...","{'name': 'persons', 'value': 'Bennett, Stetson...","{'name': 'persons', 'value': 'Stroud, C J (200...","{'name': 'persons', 'value': 'Smart, Kirby', '...","{'name': 'persons', 'value': 'Day, Ryan (1979-...",...,None,None,None,None,None,None,None,None,None,None


How about the healdine column? Can we find anything that can be useful?

In [ ]:
# Create a new DataFrame by unnesting the 'headline' column
headline_df = pd.json_normalize(full_year_data['headline'])
# Visualize the first few rows
headline_df.head()

,main,kicker,content_kicker,print_headline,name,seo,sub
0,How Do You Think About the Question of When Li...,None,None,,None,None,None
1,When Does Life Begin?,None,None,When Does Life Start? A Post-Roe Conundrum.,None,None,None
2,T.C.U. Upends Michigan in Wild Playoff Semifinal,None,None,"T.C.U., Dealing Michigan First Loss, Reaches C...",None,None,None
3,"In Bucha, a Final Rampage Served as a Coda to ...",None,None,"In Bucha, a Final Rampage Served as Coda to a ...",None,None,None
4,Georgia Downs Ohio State in Semifinal on Last-...,None,None,Razor-Thin Win in Thriller Gives Georgia Work ...,None,None,None


## Answer research Question

Geographical Representation in NYT Articles: This research question aims to explore how The New York Times (NYT) covers different regions of the world in 2023.

### Using spacy model to answer the research question

- The spaCy model looks through the abstract column in the data frame and finds the geopolitical entities (GPE)  - places, countries, cities, etc...

- Once, those GPE entities are found, we'll count the number of occurrences of those places by month.

In [ ]:
import spacy
import pandas as pd
import calendar

# Load spaCy with the English language model
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities using spaCy NER
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ == 'GPE']
# Apply NER function to 'abstract' column
full_year_data['places_mentioned'] = full_year_data['abstract'].apply(extract_entities)
# Explode DataFrame to have one row for each place mentioned
exploded_df = full_year_data.explode('places_mentioned').reset_index(drop=True)
# Extract the month from the 'pub_date' column
exploded_df['pub_month'] = pd.to_datetime(exploded_df['pub_date']).dt.month
# Group by month and place, and count occurrences
grouped_df = exploded_df.groupby(['pub_month', 'places_mentioned']).size().reset_index(name='mentions_count')
# Map numerical month values to month names
grouped_df['pub_month'] = grouped_df['pub_month'].apply(lambda x: calendar.month_name[x])

Let's view a few rows for the grouped_df

In [ ]:
grouped_df.head(10)

,pub_month,places_mentioned,mentions_count
0,January,+ Renfro,1
1,January,A.I.,4
2,January,Afghanistan,2
3,January,Alabama,6
4,January,Alameda,1
5,January,Alameda County,1
6,January,Alaska,3
7,January,Alberta,1
8,January,Albuquerque,1
9,January,Alexandria,1


The spaCy model is not perfect, we get GPE like A.I. which I doubt is the country name. However, overall, it does a good work  finding those GPE entities

Visualize the top 5 places that New York Times covers the most in its articles

In [ ]:
import plotly.express as px

# Get the top 5 places mentioned
top_5_places = grouped_df.groupby('places_mentioned')['mentions_count'].sum().nlargest(5).index
# Filter the DataFrame for the top 5 places
top_5_df = grouped_df[grouped_df['places_mentioned'].isin(top_5_places)]

top_5_df.head()

,pub_month,places_mentioned,mentions_count
73,January,China,36
155,January,Israel,11
312,January,Russia,59
369,January,U.S.,40
373,January,Ukraine,101


Plot a time series line plot

In [ ]:
# Plotting the line
fig = px.line(top_5_df, x='pub_month', y='mentions_count', color='places_mentioned',
              labels={'mentions_count': 'Mentions Count', 'pub_month': 'Month'},
              title='Top 5 Places Mentioned Over Time')

# Show the interactive plot
fig.show()

The New York Times has consistently written about Ukraine, the U.S., Russia, China, and Israel throughout the year.

 - Ukraine and Russia dominate NYT's articles due to their ongoing war.
 - The NYT covers U.S. politics while covering the tension between the U.S. and China when it comes to trade, Taiwan, etc.
 - The spike in coverage of Israel observed in October is due to the ongoing conflict between Israel and Hamas.

### Visualize the New York Times Coverage using Chloropleth

Let's create an interactive map that shows how NYT is covering the world in its articles.

In [22]:
fig1 = px.choropleth(
    grouped_df,
    locations='places_mentioned',  # Column containing country names
    locationmode='country names',
    color='mentions_count',  # Column containing values to be visualized
    hover_name='places_mentioned',  # Tooltip information
    animation_frame='pub_month',
    projection = 'natural earth',
    color_continuous_scale='Viridis',  # Color scale
    title='New York Times Coverage by Country',
)

# Show the plot
fig1.show()


## Using HugginsFace Transformers (BERT) to answer the research question

Due to the limitation of spaCy, we decided to try a different model BERT and compare the results.


Let's visualize how BERT categorize locations LOC for the first 5 articles

In [ ]:
from transformers import pipeline

first_5_articles = full_year_data.head(5).copy()
# Function to extract named entities using BERT NER
def extract_entities_bert(text):
    nlp_bert = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
    entities = nlp_bert(text)
    return [ent['word'] for ent in entities if ent['entity'] == 'I-LOC']
# Apply NER function to 'abstract' column
first_5_articles['places_mentioned'] = first_5_articles['abstract'].apply(extract_entities_bert)
# First 5 articles
first_5_articles[['abstract', 'places_mentioned']]

Now, let's expand BERT model for the full dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

# Function to extract named entities using BERT NER
def extract_entities_bert(text):
    nlp_bert = nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    entities = nlp_bert(text)
    return [ent['word'] for ent in entities if ent['entity'] == 'I-LOC']
# Apply NER function to 'abstract' column
full_year_data['places_mentioned'] = full_year_data['abstract'].apply(extract_entities_bert)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


 We ran the BERT model on the first 5 rows and it worked but when we tried on the full dataset, it was not completed even after 5 hours.

 A work around was to use the base BERT model instead of the Large language BERT model. However, that didn't complete either.

 The other solution was to use Batch Processing and Parallel processing. However, after running overnight, it didn't complete either

### Compare the model spaCy vs BERT

**BERT (Bidirectional Encoder Representations from Transformers):**

- Strengths:
Deep contextual understanding.
Versatile for various NLP tasks.
Ideal for fine-grained tasks like named entity recognition.

- Limitations:
Computationally intensive.
Resource-demanding.


---

**spaCy**:
- Strengths:
Fast and efficient.
Well-suited for tokenization and basic NLP tasks.
Lightweight and resource-friendly.
- Limitations:
Contextual understanding is based on statistical patterns.
Less suitable for tasks requiring deep contextual embeddings.

## Conclusion


In summary, the NYT's coverage reflects a combination of global events, regional conflicts, and geopolitical tensions, with an emphasis on ongoing issues in Ukraine, Russia, the U.S., China, and Israel throughout the year. The nature of the coverage aligns with major international events and conflicts shaping the geopolitical landscape.